Le but de cette partie du projet est de distinguer des phases de jeu durant le match pour ensuite déterminer durant quelle phase de jeu chaque centre a été effectué. On va s'intéresser aux phases de jeu Attaque placée cas n1 et Transition cas n1

Dans cette partie, on va étudier le type de phase de jeu numéro 1 : l'attaque placée numéro 1. Cette phase de jeu correspond à une séquence qui débute par une remise en jeu et se termine par une perte du ballon ou un arrêt du jeu.

## Attaque placée Numéro 1

In [1]:
import sqlite3

# On se connecte à la base de données générée par la partie Importation
connect = sqlite3.connect("../../Importation_donnees/raw-database.db")
cursor = connect.cursor()

# On ajoute la colonne phase_jeu si elle n'existe pas encore, sinon on la réinitialise à 0
try:
    cursor.execute("ALTER TABLE events ADD COLUMN phase_jeu INTEGER DEFAULT 0")
    connect.commit()
    print("Colonne 'phase_jeu' ajoutée.")
except sqlite3.OperationalError:
    print("La colonne 'phase_jeu' existe déjà, réinitialisation des valeurs à 0.")
    cursor.execute("UPDATE events SET phase_jeu = 0")
    connect.commit()


Colonne 'phase_jeu' ajoutée.


In [202]:
# Dans evenements_remise_en_jeu et evenement_arbitre on met tous les types d'événements qui sont considérés comme des remises en jeu.
# On a séparé les deux car les evenements_remise_en_jeu sont des pass_type alors que "Referee Ball-Drop" présent dans evenement_arbitre est présent dans la colonne type uniquement
# On a donc séparé en deux pour que ce soit plus simple de vérifier si un événement est d'un de ces types 

evenements_remise_en_jeu = {"Throw-in", "Kick Off", "Free Kick", "Corner", "Goal Kick"}
evenement_arbitre = "Referee Ball-Drop"

# On crée une table temporaire pour stocker les événements identifiés car on doit parcourir tous les événements d'une potentielle phase de jeu avant de les marquer comme faisant partie de cette phase de jeu
cursor.execute("DROP TABLE IF EXISTS phase_jeu_temp")
cursor.execute("CREATE TEMP TABLE phase_jeu_temp (event_id TEXT PRIMARY KEY);")
connect.commit()

# On récupère tous les événements dans la table events
requete = """
    SELECT event_id, match_id_SB, index_event, type AS type_evenement, pass_type, possession
    FROM events
"""
cursor.execute(requete)
evenements = cursor.fetchall()

In [203]:
# On va parcourir tous les événements (les lignes de events) pour détecter la présence de la phase de jeu numéro 1
evenements_phase = []
match_actuel = None
possession_actuelle = None
dans_une_phase = False

for i, evenement in enumerate(evenements):
    event_id, match_id, index_event, type_evenement, pass_type, possession = evenement

    # Si on change de match, on réinitialise les variables
    if match_id != match_actuel:
        match_actuel = match_id
        possession_actuelle = None
        dans_une_phase = False

    # Détection du début d'une phase de jeu (remise en jeu) dans type ou pass_type
    # On a renommé type_evenement pour plus de clarté
    if (pass_type in evenements_remise_en_jeu) or (type_evenement == evenement_arbitre):
        dans_une_phase = True
        possession_actuelle = possession

    # Si une phase est en cours, on stocke l'event_id
    if dans_une_phase and event_id is not None:
        evenements_phase.append((event_id,))

    # Un changement de possession indique une perte de ballon 
    # Fin de la phase si la possession change (on n'ajoute pas le dernier événement) ou si c'est le dernier événement du match
    if dans_une_phase and (
        (possession != possession_actuelle) or (i == len(evenements) - 1)
    ):
        # On enlève le dernier événement ajouté car il marque la fin de la phase
        evenements_phase.pop()
        dans_une_phase = False



In [204]:
# On ajoute les événements détectés dans la table temporaire
cursor.execute("DROP TABLE IF EXISTS phase_jeu_temp")
cursor.execute("CREATE TEMP TABLE phase_jeu_temp (event_id TEXT PRIMARY KEY);")
connect.commit()

if evenements_phase:
    cursor.executemany("INSERT INTO phase_jeu_temp (event_id) VALUES (?)", evenements_phase)
    connect.commit()

# On met à jour la colonne phase_jeu en mettant des 1 pour les événements concernés par cette phase de jeu
cursor.execute("""
    UPDATE events SET phase_jeu = 1 
    WHERE event_id IN (SELECT event_id FROM phase_jeu_temp);
""")
connect.commit()

cursor.execute("DROP TABLE IF EXISTS phase_jeu_temp")
connect.commit()

# On voit le nombre d'événements présents dans la phase de jeu numéro 1
print(f"{len(evenements_phase)} événements marqués avec phase_jeu = 1.")


576940 événements marqués avec phase_jeu = 1.


## Calcul du pourcentage d'événements étant dans la phase de jeu numéro 1

In [205]:
print("Le pourcentage d'événements présents dans la phase de jeu 1 est de : ", round(len(evenements_phase)/len(events)*100,2),"%")

Le pourcentage d'événements présents dans la phase de jeu 1 est de :  51.59 %


## Calcul du pourcentage de centres étant dans la phase de jeu numéro 1

In [206]:
# On récupère les événements utiles pour nos calculs
requete = """
    SELECT pass_cross, pass_type, phase_jeu 
    FROM events
    WHERE pass_cross = 1
"""
cursor.execute(requete)
evenements = cursor.fetchall()

nombre_total_centres = 0
centres_phase_1 = 0

# Parcours des événements pour compter les centres
for pass_cross, pass_type, phase_jeu in evenements:
    if pass_type != 'Corner' :
        nombre_total_centres += 1
        if phase_jeu == 1:  # On vérifie si le centre appartient à la phase de jeu 1
            centres_phase_1 += 1

# Affichage des résultats
print(f"Nombre total de centres dans events : {nombre_total_centres}")
print(f"Nombre de centres dans phase_jeu = 1 : {centres_phase_1}")

print("Le pourcentage de centres faisant partie de la phase 1 est de",round((centres_phase_1 / nombre_total_centres)*100, 2),"%")

Nombre total de centres dans events : 6946
Nombre de centres dans phase_jeu = 1 : 4140
Le pourcentage de centres faisant partie de la phase 1 est de 59.6 %


On voit que les centres sont plus représentés (59.6%) dans la phase 1 que la moyenne des événements (51.6%). On peut donc en déduire que les centres ont une forte chance de se produire durant une attaque placée (version 1)

Précision : Deux phases d'attaque placée n1 peuvent être consécutives dans le cas où la première séquence se termine par un arrêt de jeu et que la deuxième séquence aussi

---

On va à présent s'intéresser à la phase de jeu numéro 2 : la Transition cas n1. C'est une phase qui débute par une récupération du ballon et qui se termine si l’équipe reperd le ballon dans les 4 évènements suivants (hors équipe adverse). 

## Transition Numéro 1

In [207]:
import sqlite3

# On se connecte à la base de données générée par la partie Importation
connect = sqlite3.connect("../Databases/raw-database.db")
cursor = connect.cursor()

# On ajoute la colonne phase_jeu si elle n'existe pas encore, sinon on la réinitialise à 0
try:
    cursor.execute("ALTER TABLE events ADD COLUMN phase_jeu INTEGER DEFAULT 0")
    connect.commit()
    print("Colonne 'phase_jeu' ajoutée.")
except sqlite3.OperationalError:
    print("La colonne 'phase_jeu' existe déjà")
    connect.commit()

La colonne 'phase_jeu' existe déjà


In [208]:
# Définition des types d'événements pertinents pour la phase de jeu

# On choisit uniquement les événements qui concerne directement le jeu, on enlève notammenent les informations concernant les blessures, les modifications tactiques
# On ne prend également pas en compte les événements de type "Pressure", étant défensifs
types_evenements_pertinents = {
    "Pass", "Ball Receipt*", "Carry", "Dribble", "Ball Recovery",
    "Duel", "Miscontrol", "Dispossessed", "Interception", "Dribbled Past"
}

# Définition des événements considérés comme des remises en jeu
# (Même définition que pour la phase de jeu numéro 1)
evenements_remise_en_jeu = {"Throw-in", "Kick Off", "Free Kick", "Corner", "Goal Kick"}

# On récupère tous les événements de la table events
requete = """
    SELECT event_id, match_id_SB, index_event, type, pass_type, possession
    FROM events
"""
cursor.execute(requete)
evenements = cursor.fetchall()

On considère ici qu'une récupération de ballon est un changement de possession sans qu'il n'y ait remise en jeu
Une phase de Transition n1 débute donc à ce moment-là

In [209]:
# On détecte les phases de jeu correspondant à la Transition numéro 1
evenements_phase_2 = []  # Liste des identifiants d'événements à marquer avec phase_jeu = 2

i = 1
while i < len(evenements):
    evenement_actuel = evenements[i]
    id_evenement, id_match, index_evenement, type_evenement, type_passe, possession = evenement_actuel
    evenement_precedent = evenements[i - 1]
    
    # On extrait la possession de l'événement précédent pour plus de clarté
    possession_precedente = evenement_precedent[5]
    
    # On traite uniquement si les deux événements appartiennent au même match
    if id_match != evenement_precedent[1]:
        i += 1
        continue

    # On déclenche la phase dès qu'un changement de possession est détecté et que l'événement déclencheur n'est pas une remise en jeu.
    if (possession != possession_precedente) and not ((type_passe in evenements_remise_en_jeu) or (type_evenement == "Referee Ball-Drop")):
        possession_debut = possession  # Nouvelle possession
        debut_phase = i  # Cet événement marque le début de la phase
        compteur_evenements_pertinents = 0
        fin_phase = None
        
        j = i + 1
        # Parcourir les événements suivants du même match en ne comptant que ceux dont le type est dans types_evenements_pertinents
        # On ne prend donc pas en compte les événements ne figurant pas dans types_evenements_pertinents
        # C'est a dire qu'il peut y avoir plus de quatre événements durant lesquels une équipe peut se situer dans cette phase de jeu même et durant laquelle elle ne perd pas le ballon 
        # Effectivement, c'est seulement les événements pertinents qui comptent parmi les 4 événements considérés
        while j < len(evenements) and evenements[j][1] == id_match and compteur_evenements_pertinents < 4:
            evenement_suivant = evenements[j]
            id_evenement_suivant, id_match_suivant, index_evenement_suivant, type_evenement_suivant, type_passe_suivant, possession_suivante = evenement_suivant
            
            if type_evenement_suivant not in types_evenements_pertinents:
                j += 1
                continue
            
            compteur_evenements_pertinents += 1
            
            # Si la possession change par rapport à possession_debut, la phase se termine juste avant cet événement
            # On ne prend pas en compte l'événement qui fait terminer la phase car c'est le 1er événement qui ne doit pas faire partir de la phase
            if possession_suivante != possession_debut:
                fin_phase = j
                break
            
            j += 1
        
        # Valider la phase uniquement si un changement de possession a été détecté 
        if fin_phase is not None and compteur_evenements_pertinents > 0 and compteur_evenements_pertinents <= 4:
            # Marquer tous les événements de la phase, du déclencheur (debut_phase) jusqu'à fin_phase - 1
            for k in range(debut_phase, fin_phase):
                if evenements[k][1] == id_match:
                    evenements_phase_2.append(evenements[k][0])
            i = fin_phase  # Aller à l'événement déclencheur de fin de phase
            continue
    i += 1

In [210]:
# On utilise set pour éviter les doublons
evenements_phase_2 = list(set(evenements_phase_2))

# Mise à jour en base via une table temporaire
cursor.execute("DROP TABLE IF EXISTS phase1_temp")
cursor.execute("CREATE TEMP TABLE phase1_temp (event_id TEXT PRIMARY KEY);")
connect.commit()

if evenements_phase_2:
    parametres_mise_a_jour = [(id_evenement,) for id_evenement in evenements_phase_2]
    cursor.executemany("INSERT INTO phase1_temp (event_id) VALUES (?)", parametres_mise_a_jour)
    connect.commit()

cursor.execute("""
    UPDATE events SET phase_jeu = 2 
    WHERE event_id IN (SELECT event_id FROM phase1_temp);
""")
connect.commit()

cursor.execute("DROP TABLE IF EXISTS phase1_temp")
connect.commit()

print(f"{len(evenements_phase_2)} événements marqués avec phase_jeu = 2")

14340 événements marqués avec phase_jeu = 2


## Calcul du pourcentage d'événements étant dans la phase de jeu numéro 2

In [211]:
print("Le pourcentage d'événements présents dans la phase de jeu 2 est de : ", round(len(evenements_phase_2)/len(events)*100,2),"%")

Le pourcentage d'événements présents dans la phase de jeu 2 est de :  1.28 %


## Calcul du pourcentage de centres étant dans la phase de jeu numéro 2

In [212]:
# On récupère les événements utiles pour nos calculs
requete = """
    SELECT pass_cross, pass_type, phase_jeu 
    FROM events
    WHERE pass_cross = 1
"""
cursor.execute(requete)
evenements = cursor.fetchall()

nombre_total_centres = 0
centres_phase_2 = 0

# Parcours des événements pour compter les centres
for pass_cross, pass_type, phase_jeu in evenements:
    if pass_type != 'Corner':  # On exclut les corners
        nombre_total_centres += 1
        if phase_jeu == 2:  # On vérifie si le centre appartient à la phase de jeu 2
            centres_phase_2 += 1

# Affichage des résultats
print(f"Nombre total de centres dans events : {nombre_total_centres}")
print(f"Nombre de centres dans phase_jeu = 2 : {centres_phase_2}")

# Calcul du pourcentage de centres faisant partie de la phase 2
pourcentage_phase_2 = round((centres_phase_2 / nombre_total_centres) * 100, 2)
print(f"Le pourcentage de centres faisant partie de la phase 2 est de {pourcentage_phase_2} %")

Nombre total de centres dans events : 6946
Nombre de centres dans phase_jeu = 2 : 23
Le pourcentage de centres faisant partie de la phase 2 est de 0.33 %


On s'aperçoit que la phase 2 est rare (représente seulement 1.28% des événements). On observe que la répartition des centres dans cette phase de jeu est encore plus faible (0.33%). On peut donc conclure que les centres sont bien moins représentés dans cette phase de jeu (Transiton n1) que dans la moyenne des événements.

---

On s'aperçoit après vérification dans la base de données qu'aucun événement n'est à la fois dans la phase de jeu 1 et 2, ce qui est cohérent avec ce que l'on souhaite.